<a href="https://colab.research.google.com/github/ronaldsholt/ML_Vandy/blob/main/Ron_Holt_ML_Project_Idea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Background: Predicting cell lineage patterns in RNAseq measurments.

Project Description: Using representation learning as low dimentional input to higher dimentional clustering tools like TSNE or UMAP. 

Performance Metric: L2-norm/MSE